---
title: "Régression linéaire code Python"
author: "Jason Suárez"
date: "2024-02-02"
categories: ["Data Science", "Python"]
image: "image.png"
draft: false
---

Dans ce notebook, nous allons voir comment implémenter une régression linéaire en Python. Nous n'allons pas utiliser de bibliothèques pour ce faire, mais nous allons plutôt implémenter l'algorithme de régression linéaire à partir de zéro. Nous allons également voir comment utiliser notre implémentation pour prédire des valeurs à partir d'un ensemble de données.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

### Chargement et visualisation des données

In [3]:
data = pd.read_csv('data/train.csv')
data.head()

,x,y
0,24.0,21.549452
1,50.0,47.464463
2,15.0,17.218656
3,38.0,36.586398
4,87.0,87.288984
